### imports

In [1]:
from collections import defaultdict
import re

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import jaccard_score

### preprocessing data

In [2]:
df = pd.read_csv('./data/news.csv', sep=';')

In [3]:
sites = ['lenta', 'tass']

In [4]:
print(df.shape)
df.head()

(103176, 4)


,url,date,news_headline,news_body
0,https://lenta.ru/news/2019/05/22/gotepisodes/,2019-05-22T17:36:00+03:00,Названа лучшая серия «Игры престолов»,Британское издание The Guardian составило рейт...
1,https://lenta.ru/news/2019/01/09/challenge/,2019-01-09T18:16:00+03:00,Россию причислили к «внешним вызовам» для Евро...,Заместитель председателя Европейской комиссии ...
2,https://lenta.ru/news/2019/01/29/x35/,2019-01-29T19:53:00+03:00,Создатель супероружия России признал ракетные ...,Украина способна разрабатывать ракетное вооруж...
3,https://lenta.ru/news/2019/04/02/nato/,2019-04-02T10:52:00+03:00,Прибалты захотели видеть у себя больше америка...,Эстония выступает за обеспечение регулярного р...
4,https://lenta.ru/news/2019/02/01/frigate/,2019-02-01T19:05:00+03:00,Отверстие «Фрегата» устранили,"Микроотверстие, обнаруженное в российском разг..."


In [5]:
tass_df = df[(df.url.apply(lambda row: 'tass' in row))].reset_index()

In [6]:
print(tass_df.shape)
tass_df.head()

(82965, 5)


,index,url,date,news_headline,news_body
0,20156,https://tass.ru/sport/6069789,1549035126,Кремлев: МОК и AIBA не обсуждают вопрос исключ...,"АНАПА, 1 февраля. /ТАСС/. Международный олим..."
1,20157,https://tass.ru/ekonomika/1640098,1418297887,Индекс РТС в середине четверга обновил минимум...,"МОСКВА, 11 декабря. /Корр. ТАСС Елена Кожухо..."
2,20158,https://tass.ru/politika/3472158,1469028708,Яровая: антитеррористический пакет законов защ...,"МОСКВА, 20 июля. /ТАСС/. Антитеррористически..."
3,20159,https://tass.ru/ekonomika/1194085,1400363544,Hyundai отзывает с рынка США более 140 тыс. кр...,"НЬЮ-ЙОРК, 18 мая. /Корр. ИТАР-ТАСС Андрей Бе..."
4,20160,https://tass.ru/nedvizhimost/5738028,1540905058,Мутко: правительство может разработать новый м...,"МОСКВА, 30 октября. /ТАСС/. Вице-премьер РФ ..."


In [7]:
lenta_df = df[(df.url.apply(lambda row: 'lenta' in row))].reset_index()


In [8]:
print(lenta_df.shape)
lenta_df.head()

(20211, 5)


,index,url,date,news_headline,news_body
0,0,https://lenta.ru/news/2019/05/22/gotepisodes/,2019-05-22T17:36:00+03:00,Названа лучшая серия «Игры престолов»,Британское издание The Guardian составило рейт...
1,1,https://lenta.ru/news/2019/01/09/challenge/,2019-01-09T18:16:00+03:00,Россию причислили к «внешним вызовам» для Евро...,Заместитель председателя Европейской комиссии ...
2,2,https://lenta.ru/news/2019/01/29/x35/,2019-01-29T19:53:00+03:00,Создатель супероружия России признал ракетные ...,Украина способна разрабатывать ракетное вооруж...
3,3,https://lenta.ru/news/2019/04/02/nato/,2019-04-02T10:52:00+03:00,Прибалты захотели видеть у себя больше америка...,Эстония выступает за обеспечение регулярного р...
4,4,https://lenta.ru/news/2019/02/01/frigate/,2019-02-01T19:05:00+03:00,Отверстие «Фрегата» устранили,"Микроотверстие, обнаруженное в российском разг..."


In [9]:
# get only 2000 news
lenta_df = lenta_df[lenta_df.index.isin(range(2000))]

In [10]:
# get only 2000 news
tass_df = tass_df[tass_df.index.isin(range(2000))]

In [11]:
news_df = pd.concat([tass_df, lenta_df], ignore_index=True)

In [12]:
print(news_df.shape)

news_df.head()


(4000, 5)


,index,url,date,news_headline,news_body
0,20156,https://tass.ru/sport/6069789,1549035126,Кремлев: МОК и AIBA не обсуждают вопрос исключ...,"АНАПА, 1 февраля. /ТАСС/. Международный олим..."
1,20157,https://tass.ru/ekonomika/1640098,1418297887,Индекс РТС в середине четверга обновил минимум...,"МОСКВА, 11 декабря. /Корр. ТАСС Елена Кожухо..."
2,20158,https://tass.ru/politika/3472158,1469028708,Яровая: антитеррористический пакет законов защ...,"МОСКВА, 20 июля. /ТАСС/. Антитеррористически..."
3,20159,https://tass.ru/ekonomika/1194085,1400363544,Hyundai отзывает с рынка США более 140 тыс. кр...,"НЬЮ-ЙОРК, 18 мая. /Корр. ИТАР-ТАСС Андрей Бе..."
4,20160,https://tass.ru/nedvizhimost/5738028,1540905058,Мутко: правительство может разработать новый м...,"МОСКВА, 30 октября. /ТАСС/. Вице-премьер РФ ..."


In [13]:
def clear_new(new):
    return ' '.join(filter(lambda word: word != '', re.findall('[\w\d]*', new)))

In [14]:
texts_df = news_df.news_body.apply(clear_new)

In [15]:
texts_df.head()

0    АНАПА 1 февраля ТАСС Международный олимпийский...
1    МОСКВА 11 декабря Корр ТАСС Елена Кожухова Рос...
2    МОСКВА 20 июля ТАСС Антитеррористический пакет...
3    НЬЮ ЙОРК 18 мая Корр ИТАР ТАСС Андрей Бекренев...
4    МОСКВА 30 октября ТАСС Вице премьер РФ Виталий...
Name: news_body, dtype: object

### Vectorizing text

In [16]:

BOW = defaultdict(lambda: [], {})

for num, row in tqdm_notebook(enumerate(texts_df)):
    words = row.split()
    for i in range(1, len(words)):
        BOW[' '.join([words[i - 1], words[i]])].append(num)

/Users/vitaly/PycharmProjects/chelgu_nlp_labs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
arr = np.zeros((texts_df.shape[0], len(BOW)), dtype=np.uint8)

for num, shingle in enumerate(BOW):
    arr[BOW[shingle], num] = 1
    

In [18]:

shingle_indexs = []

for num, i in enumerate(arr):
    shingle_indexs.append(i.nonzero()[0])



In [20]:
print(shingle_indexs[100])

[    2    19    20   195   196   211   492   666   796   803   804  1225
  1487  2321  2615  2621  2622  2643  2703  2704  2711  2717  2720  2721
  2722  2723  2724  2726  2730  2732  2733  2734  2735  2736  2737  2738
  2739  2740  2741  2742  2743  2744  2745  2746  2747  2748  2749  2750
  2753  2754  2755  2756  2757  2758  2759  2760  2761  2762  2763  2764
  2765  2766  2776  2789  2790  2815  2921  2922  3030  3227  3245  3447
  4420  4531  4543  4875  6984  8454  8600 10091 10092 12156 12771 13963
 15352 15633 16739 18438 18948 20841 21183 22034 22035 22036 22037 22038
 22039 22040 22041 22042 22043 22044 22045 22046 22047 22048 22049 22050
 22051 22052 22053 22054 22055 22056 22057 22058 22059 22060 22061 22062
 22063 22064 22065 22066 22067 22068 22069 22070 22071 22072 22073 22074
 22075 22076 22077 22078 22079 22080 22081 22082 22083 22084 22085 22086
 22087 22088 22089 22090 22091 22092 22093 22094 22095 22096 22097 22098
 22099 22100 22101 22102 22103 22104 22105 22106 22

### Calculating a nearest pairs

In [28]:
pairs = []

for i in tqdm_notebook(range(len(shingle_indexs))):
    max_ = 0
    max_index = 0
    for j in range(i + 1, len(shingle_indexs)):
        set1 = set(shingle_indexs[i])
        set2 = set(shingle_indexs[j])
        try:
            inter = len(set1.intersection(set2))
            union = len(set1.union(set2))
            jac = inter / union
        except ZeroDivisionError:
            continue
        if jac > max_:
            max_ = jac
            max_index = j
    
    pairs.append((i, max_index, jac))
    

/Users/vitaly/PycharmProjects/chelgu_nlp_labs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [28]:
pairs

[(0, 274, 0.004640371229698376),
 (1, 1352, 0.0013123359580052493),
 (2, 3977, 0.0),
 (3, 183, 0.003663003663003663),
 (4, 1845, 0.0029940119760479044),
 (5, 1183, 0.0006891798759476223),
 (6, 100, 0.0),
 (7, 1123, 0.00980392156862745),
 (8, 266, 0.007434944237918215),
 (9, 391, 0.007194244604316547),
 (10, 163, 0.0025906735751295338),
 (11, 1231, 0.005199306759098787),
 (12, 1154, 0.0),
 (13, 690, 0.021852237252861603),
 (14, 818, 0.002044989775051125),
 (15, 1245, 0.0024600246002460025),
 (16, 220, 0.0027548209366391185),
 (17, 1277, 0.009174311926605505),
 (18, 2841, 0.01808785529715762),
 (19, 1310, 0.0023752969121140144),
 (20, 751, 0.0),
 (21, 602, 0.0),
 (22, 1620, 0.0),
 (23, 582, 0.01098901098901099),
 (24, 2577, 0.004392386530014641),
 (25, 1930, 0.007827788649706457),
 (26, 2404, 0.0),
 (27, 3208, 0.008982035928143712),
 (28, 1953, 0.008928571428571428),
 (29, 3632, 0.02262443438914027),
 (30, 642, 0.003484320557491289),
 (31, 1357, 0.0056022408963585435),
 (32, 193, 0.00236

In [111]:
texts_df[28]

'САНКТ ПЕТЕРБУРГ 10 апреля ТАСС Реестр пострадавших участников долевого строительства в Санкт Петербурге с декабря 2018 года по март 2019 года сократился почти в два раза Такие данные привел врио губернатора Санкт Петербурга Александр Беглов на встрече с главой Минстроя РФ Владимиром Якушевым в среду во время Международного арктического форума сообщили в пресс службе городской администрации Реестр пострадавших участников долевого строительства в Петербурге к марту сократился в два раза В следующем году Петербург нацелен полностью закрыть этот вопрос говорится в сообщении Проблему обманутых дольщиков городские власти решают за счет поиска новых инвесторов строительства помощи в решении юридических вопросов Комитет по строительству Петербурга и Центр содействия строительству оказывают подрядным организациям поддержку для скорейшего завершения строительства проблемных объектов Как сообщила пресс служба Минстроя РФ речь идет о периоде с 1 декабря 2018 года сейчас в реестре осталось 658 обм

In [110]:
texts_df[1953]

'САНКТ ПЕТЕРБУРГ 9 апреля ТАСС Правительство Ленинградской области зарезервировало землю под строительство второй станции метро в регионе Кудрово которую планируется ввести в эксплуатацию к концу 2025 года Об этом в кулуарах V Международного арктического форума Арктика территория диалога сообщил ТАСС во вторник губернатор Ленинградской области Александр Дрозденко Станция расположится в районе пересечения Мурманского шоссе и Кольцевой автомобильной дороги Землю мы уже зарезервировали под строительство станции Кудрово сказал он В феврале Дрозденко сообщил что власти региона также готовы профинансировать строительство наземного вестибюля станции а также благоустройство территории вокруг нее Город Кудрово расположен во Всеволожском районе и граничит с Санкт Петербургом В нем проживают до 50 тыс человек Большинство жителей Кудрова ежедневно ездят в Санкт Петербург а ближайшая станция метро Улица Дыбенко расположена в двух километрах от населенного пункта Кудрово должна стать второй станцией